In [15]:
teamname = "qcc_ducklings"
task = "part 2"
teammates = ["Jesús Fabián Macías Amador","Sebastián Domínguez","Fabián Hernández","Jesús Guzmán"]

In [1]:
import numpy as np
import pickle
import json
import os
import qiskit
from collections import Counter
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, assemble, Aer
from qiskit.visualization import plot_bloch_multivector, plot_histogram, plot_bloch_vector, array_to_latex, plot_state_qsphere
from math import pi, sqrt
import pickle

In [2]:
def normalize_image(image):
    sum_sq = 0
    for row in image:
        for pixel in row:
            sum_sq += pixel ** 2
            
    return image / np.sqrt(sum_sq)
    
def encoder(image):
    image = normalize_image(image)
    n = 10
    qc = QuantumCircuit(n)
    
    for i in range(n):
        qc.h(i)
    
    # 10 qubits, cada uno tiene 2 estados == 1024 estados a disposicion
    size = len(image)
    initial_state = np.zeros(2**n)
    counter = 0
    
    for i in range(size):
        for j in range(size):
            initial_state[counter] = image[i][j]
            counter += 1
            
    qc.initialize(initial_state)
    qc.save_statevector()
       
    return qc

def decoder(counts):
    size = 28
    reconstruction = np.zeros([size, size])

    binaries = list(counts.keys())
    decimals = list(map(lambda b: int(b, 2), binaries))

    for b in list(counts.keys()):
        d = int(b, 2)
        i = int(np.floor(d / 28))
        j = int(d % 28)
        reconstruction[i][j] = counts[b]
        
    return reconstruction

In [3]:
def histogram_to_category(histogram):
    """This function take a histogram representations of circuit execution results, and process into labels as described in 
    the problem description."""
    assert abs(sum(histogram.values())-1)<1e-8
    positive=0
    for key in histogram.keys():
        digits = bin(int(key))[2:].zfill(20)
        if digits[-1]=='0':
            positive+=histogram[key]
        
    return positive

In [4]:
#load the actual hackthon data (fashion-mnist)
images=np.load('data/images.npy')*255
labels=np.load('data/labels.npy')

In [8]:
def run_part2(image):
    # Connect to simulator
    sim = Aer.get_backend('aer_simulator')
    
    #Create a composite circuit
    classifier = pickle.load(open("save.pickle", "rb"))
    encode = encoder(image)
    circuit = encode.compose(classifier)
    
    # Assemble, simulate and get histogram
    qobj = assemble(circuit)
    result = sim.run(qobj).result()
    counts = result.get_counts()
    
    # Get label
    label = histogram_to_category(counts)
    if label > 0.51:
        label = 1
    else:
        label = 0
    return label

In [14]:
run_part2(images[4])

1024


0